<span style="font-weight:bold;color:#2d3C86;border: 1px solid black; padding: 5px;margin-left:25%"> SCRAPING AMAZON & RAKUTEN GROUPR 4: COURS SCRAPING </span>

<span style="margin-left:40%">AMAZON</span>
<img src="./amazon.jpeg" width=140px height=2px style="margin-left:37%" />
<br>
<br>
<span style="margin-left:40%">RAKUTEN</span>
<img src="./rak.png" width=140px height=240px style="margin-left:37%" />
<br>
<br>
<span style="margin-left:37%">SEARCH GROUPE4</span>
<img src="./se.png" width=240px height=640px style="margin-left:32%" />

# Importation des librairie

In [1]:
import re
import time
import uuid
import requests
from bs4 import BeautifulSoup as bs4
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor


#librairie mongo db
from pymongo import MongoClient

#generer le id de la bd
from bson.objectid import ObjectId

# Connexion à la BD

In [25]:
#connexion à MongoDB
    
server = '127.0.0.1'

port = 27017

dbName = 'groupe4'

client = MongoClient(server,port)

db = client[dbName]

product_BD = db["product"]


product_BD.drop()


# Inialisations

In [3]:
#search_word = "Livre de scraping en python"
search_word = "Livre de cuisine"


base_url = "https://www.amazon.fr"

headers = {
    
    'Connection': 'keep-alive',
    'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
    'sec-ch-ua-platform': '"macOS"',
    'Content-Type': 'text/plain;charset=UTF-8',
    'Accept': '*/*',
    'Origin': 'https://www.amazon.fr',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.amazon.fr/',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    
}


product_obj_list = []
    
page_list = []


#Variables intermediaire
page_list_ = []

page_count_ = 0

global total_produit

total_produit = 0


# Produit & Details Recherche

> CODE ASIN AMAZON : Amazon Standard Identification Number

In [4]:
product = { 
    
    "_id":None,
    "asin" : None,  
    "titre" : None, 
    "image" : None, 
    "auteur" : None , 
    "prix" : None , 
    "note" : None  , 
    "nbre_cmt" : None , 
    "format" : None,
    "best_seller":None,
    "liens":None,
    "id_search_word":None
    
}

> Session Request

In [5]:
session = requests.Session()

# Obtention des infos

## Pages à feuilleter

> Prepare URL

In [6]:

def prepare_url(base_url,search_word):
    
    url = urljoin(base_url, ("/s?k=%s" % (search_word.replace(' ', '+'))))
    
    return url



> Contenue de la page recherche

In [7]:
def get_page_content(search_url):
        
    try:
        response = session.get(base_url, headers=headers)
        
        if response.status_code != 200:
            
            raise requests.HTTPError("Erreur code:" + str(response.status_code))
        
        html = requests.get(url, headers=headers)
    
    except (requests.exceptions.ConnectionError, requests.HTTPError) as e:
        
        print(e + "Pendant la connexion à l'url: " + url)
        
        return None
    
    return html


> Obtenir nombre de pages

In [8]:
def get_page_count(page_content):
        
        soup = bs4(page_content.content , "lxml")
        
        try:
            
            pagination = soup.find_all('li', attrs={'class': ['a-normal', 'a-disabled', 'a-last']})
            
            return int(pagination[-2].text)
        
        except IndexError:
            
            return 1

In [9]:
def page_count(search_word):
    
    search_url = prepare_url(base_url,search_word)
        
    page_content = get_page_content(search_url)
        
    if (not page_content):
        
        return

    return get_page_count(page_content)

> Obtenir les urls des pages

In [10]:
def prepare_page_list(search_url,page_count):
        
    for i in range(1, page_count + 1):
        
        page_list.append(search_url + '&page=' + str(i))
        
    return page_list

## Obtentions des informations

### Obtentions des details

> Details concernant la page
- Nombre de produits par pages

In [11]:
def get_details(url, headers,search_word):
    
    try:
        response = session.get(base_url, headers=headers)
        
        if response.status_code != 200:
            
            raise requests.HTTPError("Erreur - code status: "+ str(response.status_code))
        
        #Recuperation des details
        html = requests.get(url, headers=headers)
        
        soup = bs4(html.content , "lxml")
        
        all_asin = soup.find_all('div', {'class': 'sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20'})
        
        print("Nombre Produit: "+ str(len(all_asin)))
        
        
        for i in range(len(all_asin)):
            
            product = all_asin[i].find('div', {'class': 'a-section a-spacing-medium'})
            
            #Appel de la fontion obtention des informations
            
            get_product_info(product, all_asin[i]['data-asin'],base_url,search_word)

    except (requests.exceptions.ConnectionError, requests.HTTPError) as e:
        
        print(e + "Erreur pendant la connexion à :" + url)
        
        return None
        
    return response


In [12]:
def get_product_info(div_product,asin_code, base_url,search_word):
    
    id_word = ''
    
    product["_id"] = ObjectId() 

    div_other_format = []
    
    print("Code ASIN: " + asin_code)
    
    #Insertion dans le dictionnaire des produits
    product["asin"] = asin_code
    
    if(div_product.find('img')):
        
        image = div_product.find('img')['src']
        
        print("Image : " + image)
        
        #Insertion dans le dictionnaire des produits
        product["image"] = image
        
    if(div_product.find('span', {'class': 'a-size-base-plus a-color-base a-text-normal'})):
        
        title = div_product.find('span', {'class': 'a-size-base-plus a-color-base a-text-normal'}).getText()
        
        print("Titre : "+title)
        
        #Insertion dans le dictionnaire des produits
        product["titre"] =  title
    
    if(div_product.find('a', {'class': 'a-link-normal s-no-outline'})):
        
        liens = div_product.find('a', {'class': 'a-link-normal s-no-outline'}).get('href')
        
        liens_ = base_url + liens
        
        print("Liens : "+liens_)
        
        #Insertion dans le dictionnaire des produits
        product["liens"] = liens_ 
        
        
    if(div_product.find('div', {'class': 'a-row a-size-base a-color-secondary'})):
        
        authors = []
        
        div_authors = div_product.find('div', {'class': 'a-row a-size-base a-color-secondary'}).find_all('a', {'class': 'a-size-base a-link-normal'})
        
        for i in range (len(div_authors)):
            
            authors.append(div_authors[i].getText())
            
        print("---------Auteur(s)-------")
            
        print(authors)
        
        #Insertion dans le dictionnaire des produits
        product["auteur"] = authors
            
    if(div_product.find('span', {'class': 'a-offscreen'})):
        
        price = div_product.find('span', {'class': 'a-offscreen'}).getText()
        
        print("Prix : " + price)
        
        #Insertion dans le dictionnaire des produits
        product["prix"] =  price
        
    if(div_product.find('div', {'class': 'a-row a-size-small'})):
        
        note = div_product.find('div', {'class': 'a-row a-size-small'}).find('span', {'class': 'a-icon-alt'}).getText()
        
        print("Note :" + note)
        
        #Insertion dans le dictionnaire des produits
        product["note"] = note
        
        nb_com = div_product.find('div', {'class': 'a-row a-size-small'}).find('span', {'class': 'a-size-base'}).getText()
        
        print("Nombre de commentaires : " + nb_com)
        
        #Insertion dans le dictionnaire des produits
        product["nbre_cmt"] = nb_com
        
    if(div_product.find('div', {'class': 'a-row a-size-small a-color-base'})):
        
        other_format = []
        
        if(div_product.find('div', {'class': 'a-row a-size-small a-color-base'}).find_all('a', {'class': 'a-size-base a-link-normal'})):  
            
            div_other_format = div_product.find('div', {'class': 'a-row a-size-small a-color-base'}).find_all('a', {'class': 'a-size-base a-link-normal'})
            
            for i in range (len(div_other_format)):

                other_format.append(div_other_format[i].getText())

            print("----------Format disponible ---------")

            print(other_format)
    
            #Insertion dans le dictionnaire des produits
            product["format"] = other_format
        
    if(div_product.find('span', {'class': 'a-badge-text'})):
        
        is_best_seller = True
        
        print("Oui, c'est un best seller")
        
    else :
        
        is_best_seller = False
        
        print("Non, ce n'est pas un best-seller")

   #Insertion dans le dictionnaire des produits
    product["best_seller"] =  is_best_seller
    
    id_word = search_word.replace(' ', '+')
    
    product["id_search_word"] = id_word
    
    
    
    
    
    
    
    #INSERTION DANS LA BD MONGO
    InsertedResultObj = product_BD.insert_one(product) 
    
    
    
        

> Execution des fonctions
- Le nombre de pages
- Les liens recuillirs pour le produit

In [17]:
def pre_search(search_word):
    
    #initialisation
    list_pages_produits = []
    
    nbre_page = 0
    
    search_url = prepare_url(base_url,search_word)

    page_content = get_page_content(search_url)

    if (not page_content):
        
        return
    
    #nombre de pages
    page_count_ = get_page_count(page_content)

    print("----------Nombre de page -------- ")
    print(page_count_)
                        
    #liste des pages pour le produit
    page_list_ = prepare_page_list(search_url,page_count_)
    
    print("----------Liste des pages ------- ")
    print(page_list_)
    
    
    for i in range(len(page_list_)):
        
        print("------Page "+ str(i+1) +" ------- ")
        get_details(page_list_[i], headers,search_word) 
        
    time.sleep(2)
    url = prepare_url(base_url,search_word)
    page_count = page_count(search_word)

### Obtentions des infos produits

In [19]:
pre_search(search_word)

----------Nombre de page -------- 
7
----------Liste des pages ------- 
['https://www.amazon.fr/s?k=Livre+de+cuisine&page=1', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=2', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=3', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=4', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=5', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=6', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=7', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=1', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=2', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=3', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=4', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=5', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=6', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=7', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=1', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=2', 'https://www.amazon.fr/s?k=Livre+de+cuisine&page=3', 'https://www.amazon.fr/s?k

---------Auteur(s)-------
['Jean-François Piège']
Prix : 60,00 €
Note :4,5 sur 5 étoiles
Nombre de commentaires : 158
Non, ce n'est pas un best-seller
Code ASIN: 2501151801
Image : https://m.media-amazon.com/images/I/81M6TRMh7QL._AC_UL320_.jpg
Titre : On va déguster l'Italie
Liens : https://www.amazon.fr/va-d%C3%A9guster-lItalie-Fran%C3%A7ois-R%C3%A9gis-Gaudry/dp/2501151801/ref=sr_1_22?keywords=Livre+de+cuisine&qid=1636718141&qsid=257-5162372-3220217&sr=8-22&sres=2809671834%2C2226464549%2C2263176462%2C2017138010%2C2732496162%2C231701077X%2C2017138061%2C2017138436%2CB08FKLN24H%2C2035926432%2C2263176578%2C2490334166%2C2857089260%2C2501116720%2C2013963653%2C2732498416%2C2019453665%2C2501151801%2C2012318177%2C2263172157&srpt=ABIS_BOOK
---------Auteur(s)-------
['François-Régis Gaudry']
Prix : 42,00 €
Note :4,7 sur 5 étoiles
Nombre de commentaires : 819
Oui, c'est un best seller
Code ASIN: 2012318177
Image : https://m.media-amazon.com/images/I/61Q+kvY9hXL._AC_UL320_.jpg
Titre : Le grand cou

Prix : 20,00 €
Note :4,7 sur 5 étoiles
Nombre de commentaires : 448
Non, ce n'est pas un best-seller
Code ASIN: 2732497142
Image : https://m.media-amazon.com/images/I/61M5P1vYsiL._AC_UL320_.jpg
Titre : Fait Maison - numéro 3 par Cyril Lignac
Liens : https://www.amazon.fr/Fait-Maison-num%C3%A9ro-Cyril-Lignac/dp/2732497142/ref=sr_1_53?keywords=Livre+de+cuisine&qid=1636718141&qsid=257-5162372-3220217&sr=8-53&sres=2809671834%2C2226464549%2C2263176462%2C2017138010%2C2732496162%2C231701077X%2C2017138061%2C2017138436%2CB08FKLN24H%2C2035926432%2C2263176578%2C2490334166%2C2857089260%2C2501116720%2C2013963653%2C2732498416%2C2019453665%2C2501151801%2C2012318177%2C2263172157&srpt=ABIS_BOOK
---------Auteur(s)-------
['Cyril Lignac']
Prix : 12,90 €
Note :4,8 sur 5 étoiles
Nombre de commentaires : 2 827
----------Format disponible ---------
['Format Kindle']
Oui, c'est un best seller
Code ASIN: 2016261811
Image : https://m.media-amazon.com/images/I/71ztf3ovg9L._AC_UL320_.jpg
Titre : Simplissime 2: Le

Nombre Produit: 48
Code ASIN: 237945065X
Image : https://m.media-amazon.com/images/I/51gzqmvBTrS._AC_UL320_.jpg
Titre : Le goût de Cotonou - Ma cuisine du Bénin
Liens : https://www.amazon.fr/go%C3%BBt-Cotonou-Ma-cuisine-B%C3%A9nin/dp/237945065X/ref=sr_1_101?keywords=Livre+de+cuisine&qid=1636718145&sr=8-101
---------Auteur(s)-------
['Georgiana Viou']
Prix : 28,90 €
Note :4,9 sur 5 étoiles
Nombre de commentaires : 26
Non, ce n'est pas un best-seller
Code ASIN: 2359852078
Image : https://m.media-amazon.com/images/I/81jJjflv9vL._AC_UL320_.jpg
Titre : Croq'Kilos , La Pâtisserie
Liens : https://www.amazon.fr/CroqKilos-P%C3%A2tisserie-Fanny-Comandini/dp/2359852078/ref=sr_1_102?keywords=Livre+de+cuisine&qid=1636718145&sr=8-102
---------Auteur(s)-------
['Fanny Comandini']
Prix : 12,90 €
Note :4,7 sur 5 étoiles
Nombre de commentaires : 372
Non, ce n'est pas un best-seller
Code ASIN: 2035999642
Image : https://m.media-amazon.com/images/I/71k1XhN4pLL._AC_UL320_.jpg
Titre : T12S - Transformation 

Nombre Produit: 48
Code ASIN: 2732497371
Image : https://m.media-amazon.com/images/I/41vL7k7c7qS._AC_UL320_.jpg
Titre : Poisson cru (ou presque !) - 60 recettes vertueuses issues de la pêche durable
Liens : https://www.amazon.fr/Poisson-cru-presque-recettes-vertueuses/dp/2732497371/ref=sr_1_149?keywords=Livre+de+cuisine&qid=1636718147&sr=8-149
---------Auteur(s)-------
['Carrie Solomon']
Prix : 19,90 €
Note :4,2 sur 5 étoiles
Nombre de commentaires : 4
Non, ce n'est pas un best-seller
Code ASIN: B088N62FS8
Image : https://m.media-amazon.com/images/I/51NyZ32GaOL._AC_UL320_.jpg
Titre : Mes Recettes Préférées : Cahier De Recettes Cuisine à Remplir: Livre De Recettes Personnalisé De Mes Recettes Délicieuses | Carnet De Recettes A ... |100 Fiches Recettes XXL à compléter.
Liens : https://www.amazon.fr/Mes-Recettes-Pr%C3%A9f%C3%A9r%C3%A9es-Personnalis%C3%A9-D%C3%A9licieuses/dp/B088N62FS8/ref=sr_1_150?keywords=Livre+de+cuisine&qid=1636718147&sr=8-150
---------Auteur(s)-------
[]
Prix : 3,99 €

...

Nombre Produit: 48
Code ASIN: 2732497371
Image : https://m.media-amazon.com/images/I/41vL7k7c7qS._AC_UL320_.jpg
Titre : Poisson cru (ou presque !) - 60 recettes vertueuses issues de la pêche durable
Liens : https://www.amazon.fr/Poisson-cru-presque-recettes-vertueuses/dp/2732497371/ref=sr_1_149?keywords=Livre+de+cuisine&qid=1636718178&sr=8-149
---------Auteur(s)-------
['Carrie Solomon']
Prix : 19,90 €
Note :4,2 sur 5 étoiles
Nombre de commentaires : 4
Non, ce n'est pas un best-seller
Code ASIN: B088N62FS8
Image : https://m.media-amazon.com/images/I/51NyZ32GaOL._AC_UL320_.jpg
Titre : Mes Recettes Préférées : Cahier De Recettes Cuisine à Remplir: Livre De Recettes Personnalisé De Mes Recettes Délicieuses | Carnet De Recettes A ... |100 Fiches Recettes XXL à compléter.
Liens : https://www.amazon.fr/Mes-Recettes-Pr%C3%A9f%C3%A9r%C3%A9es-Personnalis%C3%A9-D%C3%A9licieuses/dp/B088N62FS8/ref=sr_1_150?keywords=Livre+de+cuisine&qid=1636718178&sr=8-150
---------Auteur(s)-------
[]
Prix : 3,99 €

Nombre Produit: 48
Code ASIN: 2036010040
Image : https://m.media-amazon.com/images/I/71EIeAYXRlL._AC_UL320_.jpg
Titre : La Bible officielle du Cake Factory
Liens : https://www.amazon.fr/Bible-officielle-Cake-Factory/dp/2036010040/ref=sr_1_197?keywords=Livre+de+cuisine&qid=1636718179&sr=8-197
---------Auteur(s)-------
['Séverine Augé']
Prix : 18,95 €
Note :4,6 sur 5 étoiles
Nombre de commentaires : 33
----------Format disponible ---------
['Format Kindle']
Oui, c'est un best seller
Code ASIN: 2857088574
Image : https://m.media-amazon.com/images/I/718F0IUKyoL._AC_UL320_.jpg
Titre : La Patisserie de Reference - Techniques, Preparations de Base, Fiches Techniques de Fabrication
Liens : https://www.amazon.fr/Patisserie-Reference-Techniques-Preparations-Fabrication/dp/2857088574/ref=sr_1_198?keywords=Livre+de+cuisine&qid=1636718179&sr=8-198
---------Auteur(s)-------
['Jean-Michel Truchelut', 'Pierre-Paul Zeiher']
Prix : 49,00 €
Note :4,6 sur 5 étoiles
Nombre de commentaires : 156
Non, ce n'e

Nombre Produit: 48
Code ASIN: 208025085X
Image : https://m.media-amazon.com/images/I/81nAPqhqLYS._AC_UL320_.jpg
Titre : Hors piste
Liens : https://www.amazon.fr/Hors-piste-Adrien-Cachot/dp/208025085X/ref=sr_1_245?keywords=Livre+de+cuisine&qid=1636718181&sr=8-245
---------Auteur(s)-------
['Adrien Cachot']
Prix : 19,90 €
Note :4,4 sur 5 étoiles
Nombre de commentaires : 31
----------Format disponible ---------
['Format Kindle']
Non, ce n'est pas un best-seller
Code ASIN: 2036006213
Image : https://m.media-amazon.com/images/I/71cWIFI6ZmL._AC_UL320_.jpg
Titre : WW : Mes petits plats au Cookeo
Liens : https://www.amazon.fr/WW-Mes-petits-plats-Cookeo/dp/2036006213/ref=sr_1_246?keywords=Livre+de+cuisine&qid=1636718181&sr=8-246
---------Auteur(s)-------
['Ghislaine Stora']
Prix : 19,95 €
Note :4,6 sur 5 étoiles
Nombre de commentaires : 7
Non, ce n'est pas un best-seller
Code ASIN: 2263145974
Image : https://m.media-amazon.com/images/I/614-LeYi9RL._AC_UL320_.jpg
Titre : La cuisine des 5 saisons

Nombre Produit: 18
Code ASIN: B08F6RYJ6H
Image : https://m.media-amazon.com/images/I/51pBNJTkzrL._AC_UL320_.jpg
Titre : Mes 100 recettes Bistronomie - A compléter, cuisiner et savourer: Carnet, livre et cahier de cuisine à écrire, remplir & compléter soi-même I Noël I ... en petite quantité comme dans un bistrot
Liens : https://www.amazon.fr/Mes-100-recettes-Bistronomie-compl%C3%A9ter/dp/B08F6RYJ6H/ref=sr_1_293?keywords=Livre+de+cuisine&qid=1636718182&sr=8-293
---------Auteur(s)-------
[]
Prix : 10,75 €
Non, ce n'est pas un best-seller
Code ASIN: 2017134562
Image : https://m.media-amazon.com/images/I/71gkQfzWTDS._AC_UL320_.jpg
Titre : Sherlock Holmes Livre de recettes
Liens : https://www.amazon.fr/Sherlock-Holmes-Livre-recettes-Martin/dp/2017134562/ref=sr_1_294?keywords=Livre+de+cuisine&qid=1636718182&sr=8-294
---------Auteur(s)-------
[]
Prix : 19,95 €
Note :5,0 sur 5 étoiles
Nombre de commentaires : 3
Non, ce n'est pas un best-seller
Code ASIN: 2501154533
Image : https://m.media-amaz

UnboundLocalError: local variable 'page_count' referenced before assignment